In [2]:

import sys
import numpy as np
from hmmlearn.hmm import CategoricalHMM

# 狀態與觀測名稱
STATE_NAMES = ["不喜歡", "有好感", "喜歡"]
OBS_NAMES = {1: "愉快地互動", 2: "友善回應", 3: "冷漠"}

# 建立 HMM 模型
def build_model():
    # 狀態數量與觀測值數量
    n_states, n_obs = 3, 3

    # 建立 HMM 模型物件
    '''
    n_components: 隱藏狀態數量
    n_features: 觀測值數量
    init_params: 空字串表示不進行參數初始化，使用手動設定的參數
    參考文件: https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.CategoricalHMM
    '''
    model = CategoricalHMM(n_components=n_states, n_features=n_obs, init_params="")

    # 設定初始機率、轉移機率、發射機率
    # 來自狀態: 不喜歡 / 有好感 / 喜歡
    model.startprob_ = np.array([0.20, 0.60, 0.20])

    # 轉移機率矩陣與發射機率矩陣
    model.transmat_ = np.array([
        [0.72, 0.23, 0.05],
        [0.12, 0.68, 0.20],
        [0.05, 0.22, 0.73],
    ])
    model.emissionprob_ = np.array([
        [0.05, 0.18, 0.77],
        [0.25, 0.60, 0.15],
        [0.72, 0.24, 0.04],
    ])

    return model

# 小劇場生成器
def story_line(state_idx: int, obs: int, step: int):
    """
    小劇場：用 (隱狀態, 觀測) 組合生成一句描述（固定模板，不依賴 random，方便可重現）
    """
    s = STATE_NAMES[state_idx]
    o = OBS_NAMES[obs]

    # 讓敘事有點「推理感」：同一觀測在不同情緒下有不同解釋
    lines = {
        ("不喜歡", 1): [
            "她笑得很禮貌，但眼神像是在把時間切成一格一格地消耗。",
            "她的愉快像是職業反射：聲音上揚，內容卻不接球。"
        ],
        ("不喜歡", 2): [
            "她回得得體，像把話題放在托盤上遞回來，沒有多一點溫度。",
            "她很友善，但友善不等於靠近；你感覺她在維持安全距離。"
        ],
        ("不喜歡", 3): [
            "她看向窗外的時間比看向你的時間長，冷漠是一種明確訊號。",
            "她的回應短、慢、乾淨俐落，像是在結束一段不必要的對話。"
        ],
        ("有好感", 1): [
            "她主動把話題接起來，還順手丟回一顆更有趣的球。",
            "她笑出聲的瞬間很自然，像是你剛好踩到她的笑點。"
        ],
        ("有好感", 2): [
            "她的友善帶著延伸：會追問細節，也會分享自己的版本。",
            "她點頭的節奏跟你說話同步，像在說：我有在聽。"
        ],
        ("有好感", 3): [
            "她短暫冷了一下，可能是被某句話戳到雷點，也可能只是累。",
            "她沉默了一拍，但沒有逃走；更像在重新評估你這個人。"
        ],
        ("喜歡", 1): [
            "她的眼睛亮了一下，笑意不是禮貌，是失手洩漏的真心。",
            "她不只接球，還加速：把聊天變成一場雙人默契遊戲。"
        ],
        ("喜歡", 2): [
            "她很友善，而且那種友善帶著偏心：會特別記你剛剛講的小細節。",
            "她的語氣柔軟，像是已經把你放進『可以信任』的名單。"
        ],
        ("喜歡", 3): [
            "她忽然冷了一下，但不像拒絕，更像在掩飾過於明顯的在意。",
            "她把情緒收起來，可能是害羞或擔心太快被你看穿。"
        ],
    }

    # 預設句子，例如：她呈現「冷漠」，你試著推測她此刻是「喜歡」。
    # s = 狀態名稱, o = 觀測名稱
    key = (s, obs)

    # 從預設句子池中取得對應句子
    # dict.get(key, default) 的用法是: 如果 key 不在 dict 中，則回傳 default
    pool = lines.get(key, [f"她呈現「{o}」，你試著推測她此刻是「{s}」。"])

    # 用 step 來選句子，確保同樣輸入每次結果相同
    return pool[step % len(pool)]

# 美化後驗機率輸出
def pretty_probs(p):
    '''
    後驗機率指的是在觀測到某些證據後，各隱藏狀態的機率分佈。
    輸入 p 為形狀 (3,) 的 numpy 陣列，輸出格式化字串。

    p 的內容可能類似: [0.01302611 0.49758342 0.48939047]
    代表不喜歡的機率約為 1.3%，有好感約為 49.8%，喜歡約為 48.9%。
    '''
    # 將狀態名稱與機率配對，並依機率排序
    # 例如： [('不喜歡', 0.013), ('有好感', 0.498), ('喜歡', 0.489)]
    items = list(zip(STATE_NAMES, p.tolist()))
    
    # 依機率由大到小排序
    # 例如： [('有好感', 0.498), ('喜歡', 0.489), ('不喜歡', 0.013)]
    items.sort(key=lambda x: x[1], reverse=True)
    
    # 格式化輸出
    # 例如： "喜歡 0.489 / 有好感 0.498 / 不喜歡 0.013"
    return " / ".join([f"{name} {prob:.3f}" for name, prob in items])


# 主程式
if __name__ == "__main__":
    # 建立 HMM 模型
    model = build_model()

    # 使用說明
    hint = '''\
咖啡廳 HMM 推理器
- 隱藏狀態(女生真實情緒): 0不喜歡 / 1有好感 / 2喜歡
- 觀測值 (你看到的反應，請用 1/2/3 輸入):
    1 = 愉快地互動
    2 = 友善回應
    3 = 冷漠
輸入範例：1 2 2 3 2 1
離開：直接按 Enter
'''
    print(hint)

    # 主迴圈：讀取使用者輸入並進行推理
    while True:
        # 讀取觀測序列
        s = input("請輸入觀測序列(1/2/3): ").strip()

        # 離開條件
        if not s:
            print("結束。")
            break

        # 解析觀測序列
        obs_list = [int(num )for num in s.split(" ")]

        # 將觀測序列轉成 numpy 陣列 (hmmlearn 要求的格式)
        X = np.array([o - 1 for o in obs_list], dtype=int).reshape(-1, 1)

        # Viterbi：最可能的隱狀態序列
        logprob, states = model.decode(X, algorithm="viterbi")

        # Posterior：每一步各狀態的機率
        post = model.predict_proba(X)  # shape (T, 3)

        # 輸出結果摘要
        print("\n" + "-" * 72)
        print(f"你的觀測：{obs_list}  ->  {[OBS_NAMES[o] for o in obs_list]}")
        print(f"最可能心情序列：{states.tolist()}  ->  {[STATE_NAMES[i] for i in states]}")
        print(f"路徑對數似然 (log P): {logprob:.3f}")
        print("-" * 72)

        # 逐步輸出推理 + 小劇場
        for t, (o, st) in enumerate(zip(obs_list, states), start=1):
            print(f"[第{t:02d}步] 觀測={o}({OBS_NAMES[o]}) | 推測心情={st}({STATE_NAMES[st]})")
            print(f"        後驗機率: {pretty_probs(post[t-1])}")
            print(f"        小劇場: {story_line(st, o, t)}")

        # 最後一步的後驗機率分析
        last_p = post[-1]

        # 找出最高機率的狀態
        best = int(np.argmax(last_p))

        # 最高機率值
        confidence = float(last_p[best])
        
        print("-" * 72)
        
        # 總結判讀 (設定門檻為 0.8 和 0.6)
        print(f"此刻(最後一步)最可能情緒：{STATE_NAMES[best]}  (後驗={confidence:.3f})")
        if confidence >= 0.80:
            print("判讀：訊號相當明確。")
        elif confidence >= 0.60:
            print("判讀：偏向明顯，但仍可能被情境雜訊影響。")
        else:
            print("判讀：不確定性高，建議拉長觀測序列或提高觀測品質（更細的反應類別）。")
        
        print("-" * 72 + "\n")

咖啡廳 HMM 推理器
- 隱藏狀態(女生真實情緒): 0不喜歡 / 1有好感 / 2喜歡
- 觀測值 (你看到的反應，請用 1/2/3 輸入):
    1 = 愉快地互動
    2 = 友善回應
    3 = 冷漠
輸入範例：1 2 2 3 2 1
離開：直接按 Enter


------------------------------------------------------------------------
你的觀測：[1, 2, 2, 1, 1, 2, 2, 2, 1]  ->  ['愉快地互動', '友善回應', '友善回應', '愉快地互動', '愉快地互動', '友善回應', '友善回應', '友善回應', '愉快地互動']
最可能心情序列：[1, 1, 1, 1, 1, 1, 1, 1, 1]  ->  ['有好感', '有好感', '有好感', '有好感', '有好感', '有好感', '有好感', '有好感', '有好感']
路徑對數似然 (log P): -11.695
------------------------------------------------------------------------
[第01步] 觀測=1(愉快地互動) | 推測心情=1(有好感)
        後驗機率: 有好感 0.601 / 喜歡 0.382 / 不喜歡 0.017
        小劇場: 她笑出聲的瞬間很自然，像是你剛好踩到她的笑點。
[第02步] 觀測=2(友善回應) | 推測心情=1(有好感)
        後驗機率: 有好感 0.705 / 喜歡 0.275 / 不喜歡 0.020
        小劇場: 她的友善帶著延伸：會追問細節，也會分享自己的版本。
[第03步] 觀測=2(友善回應) | 推測心情=1(有好感)
        後驗機率: 有好感 0.643 / 喜歡 0.341 / 不喜歡 0.015
        小劇場: 她點頭的節奏跟你說話同步，像在說：我有在聽。
[第04步] 觀測=1(愉快地互動) | 推測心情=1(有好感)
        後驗機率: 喜歡 0.667 / 有好感 0.327 / 不喜歡 0.006
        小劇場: 她主動把話題接起來，還順手丟回一顆更有趣的球。